In [25]:
from calix.base import StatefulConnection
import calix.spiking
import quantities
import copy
from dlens_vx_v3 import sta, lola, hxcomm

In [26]:
def calibrate(target: calix.spiking.SpikingCalibTarget) -> lola.Chip:
    result = None
    with StatefulConnection() as connection:
        result = calix.spiking.calibrate(connection, target)
    dumper = sta.PlaybackProgramBuilderDumper()
    result.apply(dumper)
    result = sta.convert_to_chip(dumper.done())
    return result

In [27]:
predictive_mode = True

In [21]:
excitatory_neuron_soma_params = {
    'membrane_capacitance': 16,  # [0, 63]
    'tau_mem': quantities.quantity.Quantity(5., "us"),
    'tau_syn': (
        quantities.quantity.Quantity(5., "us"),  # exc
        quantities.quantity.Quantity(0.5, "us")),  # inh
    'leak': 60,                  # CADC lsb
    'reset': 60,                 # CADC lsb
    'threshold': 120 if predictive_mode else 75,            # CADC lsb
    'i_synin_gm': 700,           # capmem current lsb
    'refractory_time': quantities.quantity.Quantity(2., "us"),
    'synapse_dac_bias': 600,
    'holdoff_time': quantities.quantity.Quantity(0., "us")
}

excitatory_neuron_dendrite_params = {
    'membrane_capacitance': 16,  # [0, 63]
    'tau_mem': quantities.quantity.Quantity(5., "us"),  # unclear
    'tau_syn': quantities.quantity.Quantity(2., "us"),
    'leak': 60,                  # CADC lsb
    'reset': 125,                 # CADC lsb
    'threshold': 72 if predictive_mode else 68,            # CADC lsb
    'i_synin_gm': 700,           # capmem current lsb
    'refractory_time': quantities.quantity.Quantity(60., "us"),
    'synapse_dac_bias': 600,
    'holdoff_time': quantities.quantity.Quantity(15., "us")
}

inhibitory_neuron_params = {
    'membrane_capacitance': 16,  # [0, 63]
    'tau_mem': quantities.quantity.Quantity(5., "us"),
    'tau_syn': quantities.quantity.Quantity(10., "us"),
    'leak': 80,                  # CADC lsb
    'reset': 80,                 # CADC lsb
    'threshold': 125,            # CADC lsb
    'i_synin_gm': 700,           # capmem current lsb
    'refractory_time': quantities.quantity.Quantity(2., "us"),
    'synapse_dac_bias': 600,
    'holdoff_time': quantities.quantity.Quantity(0., "us")
}

In [22]:
def assemble_calib_target(
        excitatory_neuron_somas: set(),
        excitatoy_neuron_dendrites: set(),
        inhibitory_neurons: set()):
    # check no overlap
    assert len(list(excitatory_neuron_somas)
               + list(excitatory_neuron_dendrites)
               + list(inhibitory_neurons)) \
        == len(excitatory_neuron_somas) \
        + len(excitatory_neuron_dendrites) \
        + len(inhibitory_neurons)
    
    # insert into array values
    params = copy.copy(calix.spiking.neuron.NeuronCalibTarget.DenseDefault)
    for nrn in excitatory_neuron_somas:
        for name, value in excitatory_neuron_soma_params:
            params[name][nrn] = value
    for nrn in excitatory_neuron_dendrites:
        for name, value in excitatory_neuron_dendrite_params:
            params[name][nrn] = value
    for nrn in inhibitory_neurons:
        for name, value in inhibitory_neuron_params:
            params[name][nrn] = value
    return params